# LO12 P25

### [Données]

Bonjour, je vous présente ici mon projet LO12 sur [données].

Veuillez trouver ci-dessous les instructions pour exécuter le projet.

### [Installation]

1. Installer les dépendances nécessaires en utilisant pip. Assurez-vous d'avoir Python installé sur votre machine.
2. Ouvrez un terminal et naviguez jusqu'au répertoire du projet.
3. Exécutez la commande suivante pour installer les dépendances :

```
pip install -r requirements.txt
```